# Preliminaries

Importing python packages and setting display parameters

In [23]:
import numpy as np
import pandas as pd
import itertools as it
import scipy.stats as stats

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from thesis_EAfunc import *
from thesis_visfunc import *

In [24]:
plt.style.use('bmh')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.set_option('display.latex.repr', True)
pd.set_option('display.latex.longtable', True)

# Fitness Landscape Definition

In [25]:
#Problem domain
x_min = -15
x_max = 15
y_min = -15
y_max = 15

#Known minimum
x_point = -1
y_point = -1

domain = (x_min, x_max, y_min, y_max)
point = (x_point, y_point)
img_size = (8.5, 4.25)

#Problem definition
def f(x, y):
    D = 2
    alpha = 1/8
    
    x=(x-5)/6
    y=(y-5)/6
    
    a = np.abs(x ** 2 + y ** 2 - D) ** (alpha * D)
    b = ( 0.5 * (x ** 2 + y ** 2) + (x + y) ) / D
        
    return (a + b + 0.5)

In [26]:
#Testing the minimum
print(f(-1,-1))

0.0


In [27]:
#Testing the function
print(f(-1.,-1.), f(-11.,-9.), f(11.,3.), f(-6.,9.))

50.62059878583003 5.177364279021976 6.107247239602234 0.031278340140559635


# Setting up the experiment
64 Experiments
>L-> In each experiment, one set of parameters is used.
>>L-> 40 Replicates per experiment.
>>>L-> Each replicate is different due to randomness effects.

In [28]:
#starting seed
np.random.seed(42)

## Initializing data storage

In [29]:
mult_fit_cols = ['exp'] + ['pop_s'] + ['b'] + ['mut_p'] + ['mut_s'] + ['p_sel'] + ['s_sel'] + ['run', 'generation', 'fitness_min', 'fitness_max', 'fitness_mean', 'fitness_std']
multi_fit = pd.DataFrame(columns=mult_fit_cols)
multi_fit = multi_fit.infer_objects()

## Parameter space for the experiment

### Initializing

In [30]:
# Algorithm parameters
## Number of replicates, and generations per experiment
rep_n = 30
gen_f = 200

## Population size
pop_s = [10, 160]

## Parent subpopulation's selection method and size
par_selection = ['uniform','tournament_k3']
b = [0.5, 5]
par_s = [z*y for z in pop_s for y in b]

## Progeny subpopulation's size
prog_s = par_s

### Crossover Method
crossover = 'uniform'
### Mutation method, probability and size
mutation = 'random_all_gau_dis'
mut_p = [0.1, 0.9]
mut_s = [.5, 5]

## New population selection method
sur_selection = ['fitness_proportional_selection','uniform']

### 2-Level Factors encoded values

In [31]:
inputs_labels = {'pop_s' : 'Population size',
                 'b'     : 'Progeny-to-population ratio',
                 'mut_p' : 'Mutation Probability',
                 'mut_s' : 'Mutation size',
                 'p_sel' : 'Parent selection',
                 's_sel' : 'Survivor selection method'
                }

dat = [('pop_s',  10, 30, -1, 1, 'Numerical'),
       ('b'    , 0.5, 5, -1, 1, 'Numerical'),
       ('mut_p', 0.1, 0.9, -1, 1, 'Numerical (<1)'),
       ('mut_s', 0.1, 10, -1, 1, 'Numerical'),
       ('p_sel', 'fitness proportional', 'tournament k3', -1, 1, 'Categorical'),
       ('s_sel', 'fitness proportional', 'tournament k3', -1, 1, 'Categorical')
      ]

inputs_df = pd.DataFrame(dat,columns=['index', 'Value_low', 'Value_high', 'encoded_low', 'encoded_high', 'Variable type'])
inputs_df = inputs_df.set_index(['index'])
inputs_df['Label'] = inputs_df.index.map( lambda z : inputs_labels[z] )
inputs_df = inputs_df[['Label', 'Variable type', 'Value_low', 'Value_high', 'encoded_low', 'encoded_high' ]]

inputs_df

,Label,Variable type,Value_low,Value_high,encoded_low,encoded_high
index,,,,,,
pop_s,Population size,Numerical,10,30,-1,1
b,Progeny-to-population ratio,Numerical,0.5,5,-1,1
mut_p,Mutation Probability,Numerical (<1),0.1,0.9,-1,1
mut_s,Mutation size,Numerical,0.1,10,-1,1
p_sel,Parent selection,Categorical,fitness proportional,tournament k3,-1,1
s_sel,Survivor selection method,Categorical,fitness proportional,tournament k3,-1,1


### Combining the 2-level Factors

We create a list with all the possible combinations of the 2-level factors

In [32]:
exp_par = list(it.product(pop_s, b, mut_p, mut_s, par_selection, sur_selection))
print('Cantidad de combinaciones de parametros en "exp_par" :'+str(len(exp_par)))
print()
print('Primera y última combinación de parametros en "exp_par":')
print('Secuencia (pop_s, b, mut_p, mut_s, p_sel, s_sel)')
print(exp_par[0])
print(exp_par[63])

Cantidad de combinaciones de parametros en "exp_par" :64

Primera y última combinación de parametros en "exp_par":
Secuencia (pop_s, b, mut_p, mut_s, p_sel, s_sel)
(10, 0.5, 0.1, 0.1, 'fitness_proportional_selection', 'fitness_proportional_selection')
(30, 5, 0.9, 10, 'tournament_k3', 'tournament_k3')


# Experiment execution

In [10]:
%%time
exp_n = 1
for (zz, yy, xx, vv, uu, tt) in exp_par:
    sur_selection = tt
    par_selection = uu
    mut_s = vv
    mut_p = xx
    b = yy
    pop_s = zz
    prog_s = int(b * pop_s)
    par_s = prog_s
    
    fitness_res = EA_exp_only_fitness(rep_n, gen_f, f, domain, pop_s, par_s, prog_s, mut_p, mut_s, par_selection, crossover, mutation, sur_selection)
    
    fitness_res.insert(0, 's_sel', tt)
    fitness_res.insert(0, 'p_sel', uu)
    fitness_res.insert(0, 'mut_s', vv)
    fitness_res.insert(0, 'mut_p', xx)
    fitness_res.insert(0, 'b', yy)
    fitness_res.insert(0, 'pop_s', zz)
    fitness_res.insert(0, 'exp', exp_n)
    multi_fit = multi_fit.append(fitness_res, ignore_index=True, sort=False)
    multi_fit = multi_fit.infer_objects()
    
    exp_n += 1

Wall time: 1h 12min 40s


## Data storage

Writing the Data Frame to a pickle file

In [34]:
multi_fit.to_pickle('./TEST_A_2L_FitData.gz', compression='gzip')

Reading the Data Frame from a pickle file

In [33]:
multi_fit = pd.read_pickle('./TEST_A_2L_FitData.gz', compression='gzip')

# Processing data for DOE Analysis

In [35]:
multi_fit.tail()

,exp,pop_s,b,mut_p,mut_s,p_sel,s_sel,run,generation,fitness_min,fitness_max,fitness_mean,fitness_std
232315,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,116,7.193797,71.236066,45.619641,14.213987
232316,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,117,11.467743,73.680178,44.388044,15.458068
232317,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,118,7.543423,64.100916,39.732343,17.950929
232318,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,119,7.100332,66.171390,42.403105,17.924494
232319,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,120,7.100332,61.400337,41.825771,16.100210


In [23]:
# Storing the latest generation's population of each replicate
query = (multi_fit['generation']==gen_f)
multi_final_fitness_res = multi_fit[query]

# Reordering columns
multi_final_fitness_res.drop(['exp', 'generation'], axis=1, inplace=True)
multi_final_fitness_res.columns = ['pop_s', 'b', 'mut_p', 'mut_s', 'p_sel', 's_sel', 'run', 'f_min', 'f_max', 'f_mean', 'f_std']
multi_final_fitness_res = multi_final_fitness_res[['run', 'pop_s', 'b', 'mut_p', 'mut_s', 'p_sel', 's_sel', 'f_min', 'f_max', 'f_mean', 'f_std']]

# Encoding values for DOE's Factos
multi_final_fitness_res['pop_s'] = multi_final_fitness_res['pop_s'].replace([10, 30], [-1, 1]).infer_objects()
multi_final_fitness_res['b'] = multi_final_fitness_res['b'].replace([.5, 5], [-1, 1]).infer_objects()
multi_final_fitness_res['mut_p'] = multi_final_fitness_res['mut_p'].replace([.1, .9], [-1, 1]).infer_objects()
multi_final_fitness_res['mut_s'] = multi_final_fitness_res['mut_s'].replace([.1, 10], [-1, 1]).infer_objects()
multi_final_fitness_res['p_sel'] = multi_final_fitness_res['p_sel'].replace(['fitness_proportional_selection','tournament_k3'], [-1, 1]).infer_objects()
multi_final_fitness_res['s_sel'] = multi_final_fitness_res['s_sel'].replace(['fitness_proportional_selection','tournament_k3'], [-1, 1]).infer_objects()
multi_final_fitness_res = multi_final_fitness_res.set_index('run')
multi_final_fitness_res.head()

,pop_s,b,mut_p,mut_s,p_sel,s_sel,f_min,f_max,f_mean,f_std
run,,,,,,,,,,
0,-1,-1.0,-1.0,-1.0,-1,-1,5.143290,5.143290,5.143290,0.000000
1,-1,-1.0,-1.0,-1.0,-1,-1,42.836483,42.959203,42.898512,0.044530
2,-1,-1.0,-1.0,-1.0,-1,-1,45.071924,45.071924,45.071924,0.000000
3,-1,-1.0,-1.0,-1.0,-1,-1,0.024066,0.024066,0.024066,0.000000
4,-1,-1.0,-1.0,-1.0,-1,-1,17.900133,24.086404,18.518760,1.956271


In [25]:
inputs_df.to_pickle('./TEST_A_DOE_data.gz', compression='gzip')
multi_final_fitness_res.to_pickle('./TEST_A_DOE_code.gz', compression='gzip')